In [ ]:
# !pip install azure-storage-blob
# !pip install python-dotenv

import os
from dotenv import load_dotenv
from setup_utils import fetch_data, load_data, create_time_index

CONNECTION_STRING = os.getenv("CONNECTION_STRING")

load_dotenv()
# fetch_data(CONNECTION_STRING)

(
    brand_mapping,
    macro_data,
    brand_constraint,
    pack_constraint,
    segment_constraint,
    sales_data,
    volume_variation_constraint,
) = load_data()

(
    macro_data,
    sales_data,
) = create_time_index([macro_data, sales_data])



In [ ]:
import lightgbm as ltb

In [ ]:
# df = sales_data[sales_data.volume.notna()].join(macro_data, on="date", how="left")
# df_test = sales_data[sales_data.volume.isna()].join(macro_data, on="date", how="left")

df = sales_data.join(macro_data, on="date", how="left")

df = df[((df.gto>0) & (df.volume>0) & (df.promotional_discount<=0) & (df.other_discounts<=0)) | df.gto.isna() ]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
import numpy as np
# from scipy import stats

df_og = df.copy(deep=True)

# Identify and convert categorical columns to label encoding
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [ ]:
df.head(3)

In [ ]:
target_column = ["gto", "volume"]
drop_cols = ["sku","brand","pack","size","total_discounts","excise","net_revenue","maco","vilc"]
X = df[df.gto.notna()].drop(columns=[*target_column] + drop_cols)
y = df[df.gto.notna()][target_column]

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns,index=X.index)

In [ ]:
X.head()

In [ ]:
x_train = X[X.index<"2023-04-01"]
x_test = X[X.index>="2023-04-01"]

y_train = y[y.index<"2023-04-01"]
y_test = y[y.index>="2023-04-01"]

In [ ]:
# X_test = df[df.gto.isna()].drop(columns=[*target_column])

# # Define a custom scorer for Weighted Absolute Percentage Error (WAPE)
# def wape_scorer(y_true, y_pred):
#     weighted_absolute_percentage_errors = np.abs((y_true - y_pred) / y_true) * np.abs(y_pred)
#     wape = np.sum(weighted_absolute_percentage_errors) / np.sum(np.abs(y_pred))
#     return np.mean(1 - wape)  # Return 1 - WAPE as a scorer

# # Create a multi-output XGBoost model
# model = MultiOutputRegressor(xgb.XGBRegressor())

# # Define the WAPE scorer using make_scorer
# wape_scorer = make_scorer(wape_scorer, greater_is_better=True)

# # Perform 5-fold cross-validation and calculate WAPE scores
# cross_val_scores = cross_val_score(model, X, y, cv=5, scoring=wape_scorer)

# # Print WAPE scores for each fold
# for fold, score in enumerate(cross_val_scores, start=1):
#     print(f'Fold {fold}: WAPE Score = {score:.4f}')

# # Print the mean WAPE score across all folds
# mean_wape_score = cross_val_scores.mean()
# print(f'Mean WAPE Score across Folds = {mean_wape_score:.4f}')

model = MultiOutputRegressor(xgb.XGBRegressor())

# model = MultiOutputRegressor(ltb.LGBMClassifier())

# Train the model on the training data
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

In [ ]:
y_pred = pd.DataFrame(y_pred,columns = y_test.columns,index=y_test.index)

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
wape = (y_test - y_pred).abs().sum() / y_test.sum()
print(wape)

In [ ]:
import numpy as np
from scipy.optimize import minimize

def custom_objective(promo_discount, X, model, X_test, X_test_new):
    # Create a copy of X with the promo_discount column replaced
    X_modified = X.copy()
    X_modified[:, promo_column_index] = promo_discount

    # Predict using the modified X on the test set
    predictions_new = model.predict(X_test_new)

    # Predict using the modified X on the original test set
    predictions_original = model.predict(X_test)

    # Calculate the objective function value
    objective_value = -((predictions_new.sum() - predictions_original.sum()) / X_test.sum())

    return objective_value

# Replace these with your actual data and model
X = ...  # Your input data (including promo_discount column)
Y = ...  # Your target variables
X_test_new = ...  # New test data
X_test = ...  # Original test data
model = ...  # Your machine learning model

# Identify the column index for 'promo_discount' in X
promo_column_index = X.columns.get_loc('promo_discount')

# Initialize the initial guess for promo_discount
initial_guess = 0.1  # Replace with your initial value

# Define the optimization bounds for promo_discount (e.g., between 0 and 1)
bounds = [(0, 1)]

# Optimize promo_discount to maximize the objective function
result = minimize(custom_objective, initial_guess, args=(X.values, model, X_test.values, X_test_new.values),
                  bounds=bounds, method='L-BFGS-B')

# Extract the optimized promo_discount
optimized_promo_discount = result.x[0]

# Print the optimized promo_discount value
print(f'Optimized Promo Discount: {optimized_promo_discount:.4f}')


In [ ]:
df = pd.read_excel(r"C:\Users\40107702\Documents\PyMMM\mroi-ml-lib\data\hackathon\raw\hackathon_macroeconomics_data.xlsx")

In [ ]:
df.to_csv(r"C:\Users\40107702\Documents\PyMMM\mroi-ml-lib\data\hackathon\raw\hackathon_macroeconomics_data.csv",index=False)

In [ ]:
df

In [ ]:
x_train